In [24]:
%matplotlib inline

import xarray as xa
import netCDF4 as nc
import pandas as pd
import numpy as np
import urllib
import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.dates import YearLocator, WeekdayLocator, MonthLocator, DayLocator, HourLocator, DateFormatter
import matplotlib.ticker as ticker

import cmocean

In [25]:
### Plot settings
mpl.rcParams['axes.grid'] = False
mpl.rcParams['axes.edgecolor'] = 'white'
mpl.rcParams['axes.linewidth'] = 0.25
mpl.rcParams['grid.linestyle'] = '--'
mpl.rcParams['grid.linestyle'] = '--'
mpl.rcParams['xtick.major.size'] = 4
mpl.rcParams['xtick.minor.size'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['xtick.minor.width'] = 1
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['ytick.minor.size'] = 2
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['ytick.minor.width'] = 1
mpl.rcParams['ytick.direction'] = 'out'
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['ytick.color'] = 'k'
mpl.rcParams['xtick.color'] = 'k'
mpl.rcParams['font.size'] = 24
mpl.rcParams['font.sans-serif'] = "Arial"
mpl.rcParams['font.family'] = "sans-serif"
mpl.rcParams['font.weight'] = 'medium'
mpl.rcParams['svg.fonttype'] = 'none'

The following cell loads the dataset from an ERDDAP server.  It can be accessed like a opendap/thredds server for netcdf, but sends the data in a **streaming** format which is hard to figure out how to parse.  So instead, download a temporary file specifying the parameters in the url. 

Alternatives would be to access streaming version of csv file or other filetype from ERDDAP Server and process via pandas

In [47]:
ALAMOID = "http://ferret.pmel.noaa.gov/alamo/erddap/tabledap/arctic_heat_alamo_profiles_9119"
cmap = cmocean.cm.thermal
temp_filename = "data/tmp.nc"

start_date="2016-08-12"
end_date  ="2017-12-31" 
min_pres  ="0" #don't include data from when the instrument is in the air
urllib.urlretrieve(ALAMOID+".ncCFMA?profileid%2CFLOAT_SERIAL_NO%2CCYCLE_NUMBER%2CREFERENCE_DATE_TIME%2CJULD%2Ctime%2Clatitude%2Clongitude%2CPRES%2CTEMP%2CPSAL&time%3E="+start_date+"&time%3C="+end_date+"T23%3A52%3A00Z&PRES%3E="+min_pres,temp_filename)

start_date_dt = datetime.datetime.strptime(start_date,"%Y-%m-%d"),
end_date_dt   = datetime.datetime.strptime(end_date,"%Y-%m-%d")

In [48]:
#datanc = nc.Dataset('data/tmp.nc') #using netcdf library
dataxa = xa.open_dataset('data/tmp.nc')

#print dataxa

In [49]:
ds = dataxa.groupby('profile')

In [50]:
pdf = pd.DataFrame({'Temperature':dataxa.TEMP.groupby('profile').mean().data,
                'latitude':dataxa.latitude.groupby('profile').mean().data,
                'longitude':dataxa.longitude.groupby('profile').mean().data},
                index=dataxa['time'][:,0].data)

In [51]:
pdf12 = pdf.resample('12H').mean()

In [52]:
pdf12.to_csv('data/temp.csv')

In [53]:
pdf12

,Temperature,latitude,longitude
2017-09-13 12:00:00,7.281654,70.012068,-166.926626
2017-09-14 00:00:00,6.643202,70.053418,-166.840594
2017-09-14 12:00:00,6.711677,70.123396,-166.716975
2017-09-15 00:00:00,6.464200,70.197593,-166.627881
2017-09-15 12:00:00,6.306489,70.272307,-166.555682
2017-09-16 00:00:00,6.374565,70.311026,-166.514174
2017-09-16 12:00:00,6.419444,70.349789,-166.489213
2017-09-17 00:00:00,6.379500,70.386486,-166.451294
2017-09-17 12:00:00,6.334080,70.405114,-166.428989
2017-09-18 00:00:00,6.269952,70.422434,-166.434236
